## Data preparation

### Load curated dataset

In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 200)
import matplotlib.pyplot as plt
import seaborn as sn

madrid_df = pd.read_csv("data/clean_madrid_houses.csv")
df = madrid_df.copy()

display(df.sample(5))

,sq_mt_built,n_rooms,n_bathrooms,has_lift,has_parking,is_exterior,floor,energy_certificate,neighborhood,house_type,rent_price,buy_price
6140,85,2,2,True,True,True,8,unknown,Valdezarza,flat,1551,449000
4793,74,3,1,False,False,True,4,unknown,Fontarrón,flat,740,149000
15402,65,2,2,True,False,True,3,good,Goya,flat,1678,505000
11225,112,3,3,True,True,True,3,good,Castilla,flat,1781,550000
15376,360,5,6,True,True,True,5,unknown,Castellana,flat,9420,2600000


### Split data and targets

In [2]:
y_buy = df['buy_price']
y_rent = df['rent_price']
X = df.drop(['buy_price', 'rent_price'], axis='columns')

### Create dummies

In [3]:
display(X['floor'].unique())
floor_dummies = pd.get_dummies(X['floor'], prefix='floor').drop('floor_other', axis='columns')

display(X['energy_certificate'].unique())
energy_dummies = pd.get_dummies(X['energy_certificate'], prefix='energy').drop('energy_unknown', axis='columns')

display(X['neighborhood'].unique())
neighborhood_dummies = pd.get_dummies(X['neighborhood'], prefix='nbhood').drop('nbhood_Abrantes', axis='columns')

display(X['house_type'].unique())
house_dummies = pd.get_dummies(X['house_type'], prefix='htype').drop('htype_attic', axis='columns')

X = X.drop(['floor', 'energy_certificate', 'neighborhood', 'house_type'], axis='columns')
X = pd.concat([X, floor_dummies, energy_dummies, house_dummies, neighborhood_dummies], axis='columns')

X['has_lift'] = X['has_lift'].astype('uint8')
X['has_parking'] = X['has_parking'].astype('uint8')
X['is_exterior'] = X['is_exterior'].astype('uint8')

display(X)
display(X.dtypes)

array(['3', '4', '1', '0', '2', '7', '5', '6', 'other', '8', '9'],
      dtype=object)

array(['good', 'unknown', 'bad'], dtype=object)

array(['San Cristóbal', 'Los Ángeles', 'San Andrés', 'Los Rosales',
       'Butarque', 'Valdebernardo - Valderribas', 'Ambroz',
       'Casco Histórico de Vicálvaro', 'El Cañaveral - Los Berrocales',
       'Ensanche de Vallecas - La Gavia', 'Casco Histórico de Vallecas',
       'Santa Eugenia', 'Orcasitas', 'San Fermín', 'Moscardó',
       'Pradolongo', 'Almendrales', 'Zofío', '12 de Octubre-Orcasur',
       'Cuzco-Castillejos', 'Cuatro Caminos', 'Bellas Vistas',
       'Berruguete', 'Valdeacederas', 'Ventilla-Almenara', 'Niño Jesús',
       'Ibiza', 'Adelfas', 'Pacífico', 'Jerónimos', 'Estrella',
       'Palomeras sureste', 'Palomeras Bajas', 'San Diego', 'Numancia',
       'Entrevías', 'Portazgo', 'Aravaca', 'Argüelles', 'Casa de Campo',
       'Ciudad Universitaria', 'Valdemarín', 'Valdezarza', 'Fontarrón',
       'Vinateros', 'Marroquina', 'Media Legua', 'Puerta del Ángel',
       'Aluche', 'Lucero', 'Los Cármenes', 'Águilas', 'Campamento',
       'El Plantío', 'Las Tablas', 'Mont

array(['flat', 'duplex', 'attic'], dtype=object)

,sq_mt_built,n_rooms,n_bathrooms,has_lift,has_parking,is_exterior,floor_0,floor_1,floor_2,floor_3,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,energy_bad,energy_good,htype_duplex,htype_flat,nbhood_12 de Octubre-Orcasur,nbhood_Acacias,nbhood_Adelfas,nbhood_Alameda de Osuna,nbhood_Almagro,nbhood_Almendrales,nbhood_Aluche,nbhood_Ambroz,nbhood_Apóstol Santiago,nbhood_Arapiles,nbhood_Aravaca,nbhood_Argüelles,nbhood_Arroyo del Fresno,nbhood_Bellas Vistas,nbhood_Bernabéu-Hispanoamérica,nbhood_Berruguete,nbhood_Buena Vista,nbhood_Butarque,nbhood_Campamento,nbhood_Campo de las Naciones-Corralejos,nbhood_Canillas,nbhood_Casa de Campo,nbhood_Casco Histórico de Barajas,nbhood_Casco Histórico de Vallecas,nbhood_Casco Histórico de Vicálvaro,nbhood_Castellana,nbhood_Castilla,nbhood_Chopera,nbhood_Chueca-Justicia,nbhood_Ciudad Jardín,nbhood_Ciudad Universitaria,nbhood_Colina,nbhood_Comillas,nbhood_Concepción,nbhood_Conde Orgaz-Piovera,nbhood_Costillares,nbhood_Cuatro Caminos,nbhood_Cuzco-Castillejos,nbhood_Delicias,nbhood_El Cañaveral - Los Berrocales,nbhood_El Plantío,nbhood_El Viso,nbhood_Ensanche de Vallecas - La Gavia,nbhood_Entrevías,nbhood_Estrella,nbhood_Fontarrón,nbhood_Fuente del Berro,nbhood_Fuentelarreina,nbhood_Gaztambide,nbhood_Goya,nbhood_Guindalera,nbhood_Huertas-Cortes,nbhood_Ibiza,nbhood_Imperial,nbhood_Jerónimos,nbhood_La Paz,nbhood_Las Tablas,nbhood_Lavapiés-Embajadores,nbhood_Legazpi,nbhood_Lista,nbhood_Los Cármenes,nbhood_Los Rosales,nbhood_Los Ángeles,nbhood_Lucero,nbhood_Malasaña-Universidad,nbhood_Marroquina,nbhood_Media Legua,nbhood_Mirasierra,nbhood_Montecarmelo,nbhood_Moscardó,nbhood_Niño Jesús,nbhood_Nueva España,nbhood_Nuevos Ministerios-Ríos Rosas,nbhood_Numancia,nbhood_Opañel,nbhood_Orcasitas,nbhood_Pacífico,nbhood_Palacio,nbhood_Palomas,nbhood_Palomeras Bajas,nbhood_Palomeras sureste,nbhood_Palos de Moguer,nbhood_Pau de Carabanchel,nbhood_Peñagrande,nbhood_Pilar,nbhood_Pinar del Rey,nbhood_Portazgo,nbhood_Pradolongo,nbhood_Prosperidad,nbhood_Pueblo Nuevo,nbhood_Puerta Bonita,nbhood_Puerta del Ángel,nbhood_Quintana,nbhood_Recoletos,nbhood_San Andrés,nbhood_San Cristóbal,nbhood_San Diego,nbhood_San Fermín,nbhood_San Isidro,nbhood_San Juan Bautista,nbhood_San Pascual,nbhood_Sanchinarro,nbhood_Santa Eugenia,nbhood_Sol,nbhood_Timón,nbhood_Trafalgar,nbhood_Tres Olivos - Valverde,nbhood_Valdeacederas,nbhood_Valdebebas - Valdefuentes,nbhood_Valdebernardo - Valderribas,nbhood_Valdemarín,nbhood_Valdezarza,nbhood_Vallehermoso,nbhood_Ventas,nbhood_Ventilla-Almenara,nbhood_Vinateros,nbhood_Virgen del Cortijo - Manoteras,nbhood_Vista Alegre,nbhood_Zofío,nbhood_Águilas
0,64,2,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,70,3,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,94,2,2,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,64,2,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,108,2,2,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

sq_mt_built                              int64
n_rooms                                  int64
n_bathrooms                              int64
has_lift                                 uint8
has_parking                              uint8
                                         ...  
nbhood_Vinateros                         uint8
nbhood_Virgen del Cortijo - Manoteras    uint8
nbhood_Vista Alegre                      uint8
nbhood_Zofío                             uint8
nbhood_Águilas                           uint8
Length: 140, dtype: object

## Select best model

### Import models

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

### Perform GridSearchCV

In [5]:
from sklearn.model_selection import GridSearchCV, KFold
import time
import traceback

model_params = {
    'linear_regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'sgd_reg': {
        'model': SGDRegressor(),
        'params': {
            'alpha': np.logspace(-5, 1, 7),
            'eta0': np.logspace(-5, -3, 3)
        }
    },
    'kernel_ridge': {
        'model': KernelRidge(),
        'params': {
            'alpha': np.logspace(-5, 1, 7),
            'kernel': ['linear', 'rbf']
        }
    },
    'elastic_net': {
        'model': ElasticNet(),
        'params': {
            'alpha': np.logspace(-5, 1, 7),
            'l1_ratio': np.linspace(0, 1, 5),
            'selection': ['cyclic', 'random']
        }
    },
    'bayesian_ridge': {
        'model': BayesianRidge(),
        'params': {
            'alpha_1': np.logspace(-7, -5, 3),
            'alpha_2': np.logspace(-7, -5, 3),
            'lambda_1': np.logspace(-7, -5, 3),
            'lambda_2': np.logspace(-7, -5, 3)
        }
    },
    'gb_reg': {
        'model': GradientBoostingRegressor(),
        'params': {
            'learning_rate': [0.05, 0.09, 0.5, 0.9],
            'max_depth': [2, 3, 5, 10, 20],
            'min_samples_split': [2, 4, 8, 16],
            'min_samples_leaf': [1, 2, 4, 8]
        }
    },
    'svm': {
        'model': SVR(),
        'params': {
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'gamma': ['scale', 'auto'],
            'C': np.logspace(-3, 1, 5)
        }
    }
}

In [ ]:
scores_buy = []
for model, attrs in model_params.items():
    try:
        print(f"******************** Training {model} model ********************")
        reg = GridSearchCV(attrs['model'], attrs['params'], cv=KFold(shuffle=True), verbose=10)
        reg.fit(X, y_buy)
        scores_buy.append({
            'model': model,
            'best_score': reg.best_score_,
            'best_params': reg.best_params_
        })
    
    except KeyboardInterrupt:
        print(f"Ctrl+C detected during algorithm: {model}")
        break
        
    except Exception:
        print(f"Exception during algorithm: {model}")
        print(traceback.format_exc())
        continue
    
result_buy = pd.DataFrame(scores_buy, columns=['model', 'best_score', 'best_params'])
display(result_buy)

******************** Training linear_regression model ********************
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START ............................................................
[CV 1/5; 1/1] END .............................., score=0.853 total time=   0.3s
[CV 2/5; 1/1] START ............................................................
[CV 2/5; 1/1] END .............................., score=0.865 total time=   0.2s
[CV 3/5; 1/1] START ............................................................
[CV 3/5; 1/1] END .............................., score=0.856 total time=   0.2s
[CV 4/5; 1/1] START ............................................................
[CV 4/5; 1/1] END .............................., score=0.867 total time=   0.2s
[CV 5/5; 1/1] START ............................................................
[CV 5/5; 1/1] END .............................., score=0.849 total time=   0.3s
******************** Training sgd_reg model ************

/home/iago/miniconda3/envs/real_estate/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 362941460437476.75, tolerance: 517233794420.6442
  model = cd_fast.enet_coordinate_descent(


[CV 1/5; 1/70] END alpha=1e-05, l1_ratio=0.0, selection=cyclic;, score=0.859 total time=  15.0s
[CV 2/5; 1/70] START alpha=1e-05, l1_ratio=0.0, selection=cyclic................


/home/iago/miniconda3/envs/real_estate/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 353857421002746.1, tolerance: 517295697086.0504
  model = cd_fast.enet_coordinate_descent(


[CV 2/5; 1/70] END alpha=1e-05, l1_ratio=0.0, selection=cyclic;, score=0.844 total time=   4.4s
[CV 3/5; 1/70] START alpha=1e-05, l1_ratio=0.0, selection=cyclic................


...

[CV 3/5; 62/70] END alpha=10.0, l1_ratio=0.0, selection=random;, score=0.744 total time=   3.0s
[CV 4/5; 62/70] START alpha=10.0, l1_ratio=0.0, selection=random................


/home/iago/miniconda3/envs/real_estate/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 675095082102404.5, tolerance: 520259490398.22675
  model = cd_fast.enet_coordinate_descent(


[CV 4/5; 62/70] END alpha=10.0, l1_ratio=0.0, selection=random;, score=0.777 total time=   3.1s
[CV 5/5; 62/70] START alpha=10.0, l1_ratio=0.0, selection=random................


/home/iago/miniconda3/envs/real_estate/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 652757398446053.5, tolerance: 517964031443.58344
  model = cd_fast.enet_coordinate_descent(


[CV 5/5; 62/70] END alpha=10.0, l1_ratio=0.0, selection=random;, score=0.748 total time=   3.0s
[CV 1/5; 63/70] START alpha=10.0, l1_ratio=0.25, selection=cyclic...............
[CV 1/5; 63/70] END alpha=10.0, l1_ratio=0.25, selection=cyclic;, score=0.741 total time=   0.1s
[CV 2/5; 63/70] START alpha=10.0, l1_ratio=0.25, selection=cyclic...............
[CV 2/5; 63/70] END alpha=10.0, l1_ratio=0.25, selection=cyclic;, score=0.740 total time=   0.1s
[CV 3/5; 63/70] START alpha=10.0, l1_ratio=0.25, selection=cyclic...............
[CV 3/5; 63/70] END alpha=10.0, l1_ratio=0.25, selection=cyclic;, score=0.744 total time=   0.1s
[CV 4/5; 63/70] START alpha=10.0, l1_ratio=0.25, selection=cyclic...............
[CV 4/5; 63/70] END alpha=10.0, l1_ratio=0.25, selection=cyclic;, score=0.778 total time=   0.1s
[CV 5/5; 63/70] START alpha=10.0, l1_ratio=0.25, selection=cyclic...............
[CV 5/5; 63/70] END alpha=10.0, l1_ratio=0.25, selection=cyclic;, score=0.748 total time=   0.1s
[CV 1/5; 64/70

In [ ]:
scores_rent = []
for model, attrs in model_params.items():
    try:
        print(f"******************** Training {model} model ********************")
        reg = GridSearchCV(attrs['model'], attrs['params'], cv=KFold(shuffle=True), verbose=10)
        reg.fit(X, y_rent)
        scores_rent.append({
            'model': model,
            'best_score': reg.best_score_,
            'best_params': reg.best_params_
        })
    
    except KeyboardInterrupt:
        print(f"Ctrl+C detected during algorithm: {model}")
        break
        
    except Exception:
        print(f"Exception during algorithm: {model}")
        print(traceback.format_exc())
        continue
    
result_rent = pd.DataFrame(scores_rent, columns=['model', 'best_score', 'best_params'])
display(result_rent)